# Schedule recordings using saved orbital data

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
library(lubridate)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



In [3]:
library(yaml)

Source to functions in Functions.ipynb 

In [4]:
eval(parse(text = system2('jupyter', c('nbconvert', 'Functions.ipynb', '--to=script', '--stdout'), stdout = TRUE)))

In [5]:
paste("Running at ", Sys.time() %>% .POSIXct("GMT"), "GMT")

[1] "Running at  2019-09-29 14:19:44 GMT"

In [35]:
config_list <- tryCatch({
    read_yaml("configuration.yaml")
}, warning = function(w) {
    print("No configuration data found. Run Calibrate_receiver.ipynb")
    stop()
}, error = function(e) {
    print("No configuration data found. Run Calibrate_receiver.ipynb")
    stop()
    }
)

In [8]:
satellite_data_df <- tryCatch({
    readRDS("satellite_data_df.rds")
}, warning = function(w) {
    print("No orbital data found. Run Satellite_orbital_data.ipynb")
    stop()
}, error = function(e) {
    print("No orbital data found. Run Satellite_orbital_data.ipynb")
    stop()
    }
)

## Convert to satellite passes each day

Get the current time in UTC

In [9]:
current_time_utc <- Sys.time() %>% .POSIXct("GMT")

Make sure we have a future satellite pass in the data, then filter out any past passes.

In [17]:
stopifnot(nrow(satellite_data_df %>% filter(startUTC > current_time_utc)) > 0)

Get rid of old passes

In [18]:
satellite_data_df <- satellite_data_df %>% filter(startUTC > current_time_utc)

## Format systemd

For systemd, using `systemd-run` rather than the usual template based form since these are one-off non-recurring commands. The argument `--user` runs the command as the user who scheduled it. `--unit` gives the job a name. The argument `--on-calendar` schedules the start and needs to be one minute before the satellite pass to give systemd and mlrpt time to start up. The format is yyyy-mm-dd HH:MM in **local time**. Documentation at https://www.freedesktop.org/software/systemd/man/systemd-run.html

## Format `rtl_fm`

rtl_fm -M raw -s 140000 -f 137.9M -E dc -g <gain> -p <ppm>

http://kmkeen.com/rtl-demod-guide/

`timeout $duration rtl_fm $biast -f $freq -s $sample -g $dongleGain -F 9 -A fast -E offset -p $dongleShift $recdir/$fileNameCore.raw | tee -a $logFile`

rtl_fm will record indefinitely. Use `timeout` to set a recording duration. **Append `m` to the time for minutes.** Seconds is the default

`-M raw` Raw output, 2x16 bit IQ pairs

`-s` Sample rate, width of recorded signal - should include few kHz for doppler shift.
sample_rate='48000'

`-g` Dongle gain. Run `rtl_test` to get supported gain levels. 49.6 seems to be the highest supported by rtl-sdr.
dongleGain='49.6'

`-F` fir_size (default: off) enables low-leakage downsample filterEnable a higher quality downsampling FIR than the default boxcar filter. `-F 0` is okay to use while `-F 9` is still a work in progress.
FIR=9

`-A` Choose how arctan is computed. Options select between the standard (floating) lib *std*, a fast polynomial integer approximation *fast* and a precomputed look-up-table *lut*
arctan=fast

`-p` Error. Correct for the parts-per-million error in the crystal. Run `Calibrate_receiver.ipynb` to calculate.
dongleshift=clock_offset

`-E` Use multiple -E options to enable multiple features.
    1) `-E dc` dc blocking filter
    2) `-E deemp` de-emphasis filter for WBFM
    3) `-E direct` direct sampling mode
    4) `-E offset` offset tuning mode (E4000 only)
    
`-T` To turn bias-t power on (i.e., for LNA) use the `-T` flag 

In [74]:
generateRecordingCommand <- function(df, config_list) {
    df <- df %>% mutate(recording_command = 
        case_when(satellite == "METEOR-M2" ~ 
            paste0('systemd-run --user --unit=\"rtl_fm_', localStartDate, "_", localStartTime,
                '\" --on-calendar=\"', localStartDate," ", localStartTime,
                '\" /usr/local/bin/mlrpt -s ',
                ' -t ', duration),
            satellite == "METEOR-M2-2" ~ 
                paste0('systemd-run --user --unit=\"rtl_fm_', localStartDate, "_", localStartTime,
            '\" --on-calendar=\"', localStartDate," ", localStartTime,
            '\" /usr/local/bin/mlrpt -s ',
            ' -t ', duration),
            startsWith(as.character(satellite), "NOAA") ~ 
                paste0('systemd-run --user --unit=\"', satellite, '_', localStartDate, '_', df$localStartTime, 
                '\" --on-calendar=\"', localStartDate," ", 
                localStartTime, '\" ', 'timeout ', duration, 'm rtl_fm -M raw -s 48000 -A fast -f ', 
                frequency, ' -g ', config_list$gain, ' -p ', config_list$clock_offset,
                ' ', config_list$recording_directory, '/', localStartDate, '_', 
                localStartTime, '.raw')))
    return(df)
}

In [76]:
satellite_data_df <- satellite_data_df %>% generateRecordingCommand(config_list)

Should try medet https://www.reddit.com/r/RTLSDR/comments/cvcqg9/i_finally_added_meteor_m22_support_to_medet/ https://github.com/artlav/meteor_decoder https://github.com/dbdexter-dev/meteor_demod https://www.reddit.com/r/RTLSDR/comments/abn29d/automatic_meteor_m2_reception_on_linux_using_rtl/  instead

In [77]:
satellite_data_df$recording_command

[1] "systemd-run --user --unit=\"NOAA_19_2019-09-29_16:41\" --on-calendar=\"2019-09-29 16:41\" timeout 15m rtl_fm -M raw -s 48000 -A fast -f 137100000 -g 49.6 -p -0.546 ~/recordings/2019-09-29_16:41.raw"
  [2] "systemd-run --user --unit=\"NOAA_19_2019-09-29_18:22\" --on-calendar=\"2019-09-29 18:22\" timeout 15m rtl_fm -M raw -s 48000 -A fast -f 137100000 -g 49.6 -p -0.546 ~/recordings/2019-09-29_18:22.raw"
  [3] "systemd-run --user --unit=\"NOAA_15_2019-09-29_18:41\" --on-calendar=\"2019-09-29 18:41\" timeout 15m rtl_fm -M raw -s 48000 -A fast -f 137620000 -g 49.6 -p -0.546 ~/recordings/2019-09-29_18:41.raw"
  [4] NA                                                                                                                                                                                                     
  [5] "systemd-run --user --unit=\"NOAA_18_2019-09-29_20:23\" --on-calendar=\"2019-09-29 20:23\" timeout 15m rtl_fm -M raw -s 48000 -A fast -f 137912500 -g 49.6 -p -0.546 ~/recordings/2019-09-29_20:23.raw"
  [6] NA                                                                                                                                                                                                     
  [7] "systemd-run --user --unit=\"NOAA_18_2019-09-29_22:04\" --on-calendar=\"2019-09-29 22:04\" timeout 14m rtl_fm -M raw -s 48000 -A fast -f 137912500 -g 49.6 -p -0.546 ~/recordings/2019-09-29_22:04.raw"
  [8] NA                                                                                                                                                                                                     
  [9] "systemd-run --user --unit=\"NOAA_19_2019-09-30_04:01\" --on-calendar=\"2019-09-30 04:01\" timeout 15m rtl_fm -M raw -s 48000 -A fast -f 137100000 -g 49.6 -p -0.546 ~/recordings/2019-09-30_04:01.raw"
 [10] "systemd-run --user --unit=\"NOAA_19_2019-09-30_05:42\" --on-calendar=\"2019-09-30 05:42\" timeout 15m rtl_fm -M raw -s 48000 -A fast -f 137100000 -g 49.6 -p -0.546 ~/recordings/2019-09-30_05:42.raw"
 [11] "systemd-run --user --unit=\"NOAA_15_2019-09-30_05:55\" --on-calendar=\"2019-09-30 05:55\" timeout 15m rtl_fm -M raw -s 48000 -A fast -f 137620000 -g 49.6 -p -0.546 ~/recordings/2019-09-30_05:55.raw"
 [12] NA                                                                                                                                                                                                     
 [13] "systemd-run --user --unit=\"NOAA_15_2019-09-30_07:35\" --on-calendar=\"2019-09-30 07:35\" timeout 14m rtl_fm -M raw -s 48000 -A fast -f 137620000 -g 49.6 -p -0.546 ~/recordings/2019-09-30_07:35.raw"
 [14] "systemd-run --user --unit=\"NOAA_18_2019-09-30_07:43\" --on-calendar=\"2019-09-30 07:43\" timeout 15m rtl_fm -M raw -s 48000 -A fast -f 137912500 -g 49.6 -p -0.546 ~/recordings/2019-09-30_07:43.raw"
 [15] NA                                                                                                                                                                                                     
 [16] "systemd-run --user --unit=\"NOAA_18_2019-09-30_09:24\" --on-calendar=\"2019-09-30 09:24\" timeout 14m rtl_fm -M raw -s 48000 -A fast -f 137912500 -g 49.6 -p -0.546 ~/recordings/2019-09-30_09:24.raw"
 [17] NA                                                                                                                                                                                                     
 [18] "systemd-run --user --unit=\"NOAA_19_2019-09-30_16:29\" --on-calendar=\"2019-09-30 16:29\" timeout 15m rtl_fm -M raw -s 48000 -A fast -f 137100000 -g 49.6 -p -0.546 ~/recordings/2019-09-30_16:29.raw"
 [19] "systemd-run --user --unit=\"NOAA_19_2019-09-30_18:10\" --on-calendar=\"2019-09-30 18:10\" timeout 15m rtl_fm -M raw -s 48000 -A fast -f 137100000 -g 49.6 -p -0.546 ~/recordings/2019-09-30_18:10.raw"
 [20] "systemd-run --user --unit=\"NOAA_15_2019-09-30_18:16\" --on-calendar=\"2019-09-30

# Get non-overlapping passes to schedule

In [89]:
satellite_data_df %>% filter(endUTC > lead(startUTC) | lag(endUTC) > startUTC) %>% nrow

[1] 36

In [87]:
rep(1:(nrow(.) / 2), each=2)

[1] 1 1 2 2 3 3 4 4 5 5 6 6 7 7

In [96]:
seq(6, 3)

[1] 6 5 4 3

In [114]:
passesToDrop(satellite_data_df)

satellite,frequency,norad_id,priority,startAz,startAzCompass,startUTC,maxAz,maxAzCompass,maxEl,maxUTC,endAz,endAzCompass,endUTC,localStartDate,localStartTime,duration,recording_command,group
<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<dttm>,<dbl>,<chr>,<dbl>,<int>,<dbl>,<chr>,<dttm>,<chr>,<chr>,<dbl>,<chr>,<int>
NOAA_15,137620000,25338,3,35.60,NE,2019-09-30 03:55:20,108.00,ESE,35.54,1569816150,180.62,S,2019-09-30 04:09:40,2019-09-30,05:55,15,"systemd-run --user --unit=""NOAA_15_2019-09-30_05:55"" --on-calendar=""2019-09-30 05:55"" timeout 15m rtl_fm -M raw -s 48000 -A fast -f 137620000 -g 49.6 -p -0.546 ~/recordings/2019-09-30_05:55.raw",1
NOAA_15,137620000,25338,3,340.05,NNW,2019-09-30 05:35:30,276.44,W,20.98,1569822135,212.33,SW,2019-09-30 05:49:00,2019-09-30,07:35,14,"systemd-run --user --unit=""NOAA_15_2019-09-30_07:35"" --on-calendar=""2019-09-30 07:35"" timeout 14m rtl_fm -M raw -s 48000 -A fast -f 137620000 -g 49.6 -p -0.546 ~/recordings/2019-09-30_07:35.raw",2
NOAA_15,137620000,25338,3,149.02,SE,2019-09-30 16:16:45,83.91,E,22.35,1569860615,18.56,NNE,2019-09-30 16:30:20,2019-09-30,18:16,14,"systemd-run --user --unit=""NOAA_15_2019-09-30_18:16"" --on-calendar=""2019-09-30 18:16"" timeout 14m rtl_fm -M raw -s 48000 -A fast -f 137620000 -g 49.6 -p -0.546 ~/recordings/2019-09-30_18:16.raw",3
NOAA_15,137620000,25338,3,49.98,NE,2019-10-01 03:31:00,111.89,ESE,20.23,1569901060,172.90,S,2019-10-01 03:44:15,2019-10-01,05:31,14,"systemd-run --user --unit=""NOAA_15_2019-10-01_05:31"" --on-calendar=""2019-10-01 05:31"" timeout 14m rtl_fm -M raw -s 48000 -A fast -f 137620000 -g 49.6 -p -0.546 ~/recordings/2019-10-01_05:31.raw",4
METEOR_M2,137100000,40069,4,27.45,NNE,2019-10-04 05:32:25,104.90,E,49.93,1570167595,184.79,S,2019-10-04 05:47:30,2019-10-04,07:32,16,NA,5
METEOR_M2,137100000,40069,4,155.89,SSE,2019-10-04 17:56:30,82.14,E,34.64,1570212235,8.10,N,2019-10-04 18:11:10,2019-10-04,19:56,15,NA,6
METEOR_M2,137100000,40069,4,38.50,NE,2019-10-05 05:13:00,108.58,ESE,31.63,1570252810,178.81,S,2019-10-05 05:27:20,2019-10-05,07:13,15,NA,7
METEOR_M2,137100000,40069,4,148.90,SE,2019-10-05 17:37:00,84.54,E,21.76,1570297435,19.71,NNE,2019-10-05 17:50:45,2019-10-05,19:37,14,NA,8
METEOR_M2,137100000,40069,4,49.93,NE,2019-10-06 04:53:45,110.85,ESE,20.41,1570338025,172.83,S,2019-10-06 05:07:10,2019-10-06,06:53,14,NA,9


# Get passes that occur during daylight

Check what's already scheduled

In [81]:
system("systemctl list-timers --user -all", intern=TRUE)

[1] "NEXT                          LEFT     LAST PASSED UNIT                           ACTIVATES"                       
[2] "Mon 2019-09-30 05:55:00 SAST  10h left n/a  n/a    NOAA_15_2019-09-30_05:55.timer NOAA_15_2019-09-30_05:55.service"
[3] "Mon 2019-09-30 07:35:00 SAST  12h left n/a  n/a    NOAA_15_2019-09-30_07:35.timer NOAA_15_2019-09-30_07:35.service"
[4] ""                                                                                                                  
[5] "2 timers listed."